In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
import pandas as pd
import statsmodels.api as sm

import sys
sys.path.append("../")
from bargains_linear import * 

In [2]:
def print_test(data):
    model1 = sm.OLS(data['phi2'],sm.add_constant(data[['phi1','c1']]))
    param, se = model1.fit().params.rename('param'),model1.fit().bse.rename('se')
    return pd.merge(param,se,left_index=True,right_index=True)

def run_test(data):
    regression_result = print_test(data)
    test_statistic = regression_result.loc['c1','param']/regression_result.loc['c1','se']
    test_result = (test_statistic > 1.96)
    return test_result

results = pd.DataFrame(columns=['dgp','test1','test2'])

for j in range(6):
    seq_result = pd.read_csv('fake_data/seq_data_%s.csv'%j,index_col=0)
    active_result = pd.read_csv('fake_data/active_data_%s.csv'%j,index_col=0)
    passive_result = pd.read_csv('fake_data/passive_data_%s.csv'%j,index_col=0)
    
    seq_params_df = pd.read_csv('fake_data/seq_params_mc_%s.csv'%j,index_col=0)
    active_params_df = pd.read_csv('fake_data/active_param_mc_%s.csv'%j,index_col=0)
    passive_params_df = pd.read_csv('fake_data/passive_params_mc_%s.csv'%j,index_col=0)


    test1p = 0
    test2p = 0
    test3p = 0
    nobs = 1000
    for i in range(nobs):
    
        subsample1 = seq_result.sample(frac=0.1)
        test1p = test1p + run_test(subsample1)
        
        subsample2 = active_result.sample(frac=0.1)
        test2p = test2p + run_test(subsample2)
        
        subsample3 = passive_result.sample(frac=0.1)
        test3p = test3p + run_test(subsample3)
        
    test1j=(seq_params_df['obj']>8).mean()
    test2j=(active_params_df['obj']>8).mean()
    test3j=(passive_params_df['obj']>8).mean()
    
    row = pd.DataFrame(columns=['dgp','test1','test2'],
        data=[['sequential',test1p/nobs,test1j],
             ['active',test2p/nobs,test2j],
             ['passive',test3p/nobs,test3j]])
    
    results = pd.concat( [results, row] )
    
print(results)
results.to_csv('mc_results.csv')

          dgp  test1     test2
0  sequential   0.00  0.000000
1      active   1.00  1.000000
2     passive   1.00  1.000000
0  sequential   0.06  0.000000
1      active   1.00  0.700000
2     passive   1.00  0.800000
0  sequential   1.00  0.133333
1      active   0.98  0.600000
2     passive   1.00  0.666667
0  sequential   0.00  0.100000
1      active   1.00  0.550000
2     passive   1.00  0.600000
0  sequential   0.01  0.080000
1      active   1.00  0.640000
2     passive   1.00  0.680000
0  sequential   0.00  0.200000
1      active   1.00  0.700000
2     passive   1.00  0.733333
